In [ ]:
import os
import dlib
import glob

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import numpy as np
%matplotlib inline

In [ ]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
import keras
#from tensorflow import keras

print(tf.__version__)
print(keras.__version__)
print(tf.config.experimental.list_physical_devices())

In [ ]:
from keras_video_datagen import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator()

train_data = datagen.flow_from_directory('data/train', 
                                         target_size=(70, 140), 
                                         batch_size=5, 
                                         frames_per_step=13, 
                                         shuffle=False, 
                                         color_mode='rgb')

val_data = datagen.flow_from_directory('data/validation', 
                                       target_size=(70, 140), 
                                       batch_size=5, 
                                       frames_per_step=13, 
                                       shuffle=False, 
                                       color_mode='rgb')

In [ ]:
x, y = train_data.next()

print(x.shape)
print(y.shape)

print(y[0])

In [ ]:
import math

print(math.ceil(13559/(5*13)))
print(math.ceil((2366/(5*13))))

In [ ]:
from keras.layers import *
from keras.regularizers import *
from keras.constraints import *

In [ ]:
model_small = keras.models.Sequential()

model_small.add(ZeroPadding3D(padding=(1, 2, 2), input_shape=(13, 70, 140, 3)))
model_small.add(Conv3D(32, (3, 5, 5), strides=(1, 2, 2), use_bias=False))
model_small.add(BatchNormalization(momentum=0.99))
model_small.add(Activation('relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

model_small.add(ZeroPadding3D(padding=(1, 2, 2)))
model_small.add(Conv3D(64, (3, 5, 5), strides=(1, 1, 1), use_bias=False))
model_small.add(BatchNormalization(momentum=0.99))
model_small.add(Activation('relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

model_small.add(ZeroPadding3D(padding=(1, 1, 1)))
model_small.add(Conv3D(96, (3, 3, 3), strides=(1, 2, 2), use_bias=False))
model_small.add(BatchNormalization(momentum=0.99))
model_small.add(Activation('relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

model_small.add(TimeDistributed(Flatten()))

model_small.add(Bidirectional(GRU(65, return_sequences=False, activation='tanh')))

model_small.add(Dense(10, activation='softmax'))

In [ ]:
# adam = keras.optimizers.Adam(lr=1e-4)
sgd = keras.optimizers.SGD(lr=4e-5, momentum=0.9, nesterov=True)
save_weights = keras.callbacks.ModelCheckpoint('model_small.hdf5', save_best_only=True, monitor='loss', mode='min')
model_small.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# model_small.summary()

In [ ]:
import os
try:
    os.remove('model_small.hdf5')
    print('file removed')
except:
    print('file is not there')

In [ ]:
history = model_small.fit_generator(train_data, epochs=100, steps_per_epoch=209, 
                                    validation_data=val_data, validation_steps=37, callbacks=[save_weights], shuffle=False)

## Plot results

In [ ]:
# !pip install seaborn

import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,7))

fig.set_facecolor("white")
fig.suptitle((
    'SGD(lr=4e-5, momentum=0.9, nesterov=True)\n\
    loss: 0.3538 - accuracy: 0.9664 - val_loss: 1.4803 - val_accuracy: 0.2747'), fontsize=16, color='black', backgroundcolor='white')

ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_xticks(np.arange(0, len(history.history['accuracy']) + 10, 10))
ax1.set_yticks(np.arange(0, 1.1, 0.1))
ax1.tick_params(axis='both', colors='black')

ax1.set_ylabel('Acuracy')
ax1.set_xlabel('Epochs')
ax1.yaxis.label.set_color('black')
ax1.yaxis.label.set_fontsize(14)
ax1.xaxis.label.set_color('black')
ax1.xaxis.label.set_fontsize(14)

ax1.grid(axis = 'both')
ax1.legend(['train', 'test'], loc='upper left')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_xticks(np.arange(0, len(history.history['loss']) + 10, 10))
ax2.set_yticks(np.arange(0, max(history.history['loss']) + 0.5, 0.3))
ax2.tick_params(axis='both', colors='black')

ax2.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.yaxis.label.set_color('black')
ax2.yaxis.label.set_fontsize(14)
ax2.xaxis.label.set_color('black')
ax2.xaxis.label.set_fontsize(14)

ax2.grid(axis = 'both')
ax2.legend(['train', 'test'], loc='upper left')

fig.show()

In [ ]:
val_data2 = datagen.flow_from_directory('data/validation', 
                                       target_size=(70, 140), 
                                       batch_size=1, 
                                       frames_per_step=13, 
                                       shuffle=False, 
                                       color_mode='rgb')

val_classes = []

for i in list(range(0, int(len(val_data2.classes)), 13)):
    val_classes.append(int(np.mean(np.array(val_data2.classes[i:i+13]))))

In [ ]:
print(math.ceil((2366/(1*13))))

In [ ]:
Y_pred = model_small.predict_generator(val_data2, 182)
y_pred = np.rint(np.argmax(Y_pred, axis=1))

In [ ]:
target_names = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
target_names = target_names.split(', ')

fig, ax1 = plt.subplots(1, 1, figsize=(10, 8))

array = confusion_matrix(val_classes, y_pred)

df_cm = pd.DataFrame(array, index = [i for i in target_names],
                  columns = [i for i in target_names])

ax1 = sn.heatmap(df_cm, annot=True)
fig.set_facecolor("white")
fig.suptitle((
    'SGD(lr=4e-5, momentum=0.9, nesterov=True)\n\
    loss: 0.3538 - accuracy: 0.9664 - val_loss: 1.4803 - val_accuracy: 0.2747'), fontsize=16, color='Black')
fig.show()

In [ ]:
print(classification_report(val_classes, y_pred, target_names=target_names))